IPL MATCH PREDICTION


# STEP1 :  DATA PREPROCESSING

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("matches.csv")
df.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [3]:
print(df.shape)
print(df.isnull().sum())


(756, 18)
id                   0
Season               0
city                 7
date                 0
team1                0
team2                0
toss_winner          0
toss_decision        0
result               0
dl_applied           0
winner               4
win_by_runs          0
win_by_wickets       0
player_of_match      4
venue                0
umpire1              2
umpire2              2
umpire3            637
dtype: int64


REMOVING UNNECESSARY ROWS THAT DON'T AFFECT THE OUTPUT LABEL


In [4]:
"""
    Removing unnecessary Columns
"""
# Umpires dont affect the match result
df.drop("umpire1",inplace=True,axis = 1)
df.drop("umpire2",inplace=True,axis = 1)
df.drop("umpire3",inplace=True,axis = 1)


#df.drop("dl_applied",inplace=True,axis = 1)
df.drop("Season",inplace=True,axis = 1)
df.drop("venue",inplace=True,axis = 1)

#These column are indirectly linked to the output and label and leads to data leaking
df.drop("win_by_wickets",inplace=True,axis = 1)
df.drop("player_of_match",inplace=True,axis = 1)
df.drop("win_by_runs",inplace=True,axis = 1)
df.drop("date",inplace=True,axis = 1)

print(df.shape)
df.head()

(756, 9)


,id,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner
0,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad
1,2,Pune,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant
2,3,Rajkot,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders
3,4,Indore,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab
4,5,Bangalore,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore


In [5]:
"""
Removing the result where Results is no result.
"""

print(df['result'].unique())
df.where(df['result'] != 'no result',inplace=True)
print(df['result'].unique())
df['result'].unique()
df.isnull().sum()
df = df[df['result'].notna()]
df.isnull().sum()

['normal' 'tie' 'no result']
['normal' 'tie' nan]


id               0
city             7
team1            0
team2            0
toss_winner      0
toss_decision    0
result           0
dl_applied       0
winner           0
dtype: int64

In [6]:
"""
Check whether team1, team2 and winner have same value or not
"""

print(df['result'].unique() )
print(df.shape)
list1 = list(df['team1'].unique())
list2 = list(df['team2'].unique())
list3 = list(df['winner'].unique())
list1.sort()
list2.sort()
list3.sort()
if list1 == list2 and list1 == list3:
    print(True)
else:
    print(False)
df.drop("result",inplace=True,axis = 1)

['normal' 'tie']
(752, 9)
True


In [7]:
"""
Conversion Of Winner Team and Toss Team to Integer
Team1 : indicates 0
Team2 : indicates 1
"""

result = []
for ind in df.index:
    if df['winner'][ind]==df['team1'][ind]:
        result.append(0)
    else:
        result.append(1)
        
df.insert(8,'match_winner',result)


winner_of_toss=[]
for ind in df.index:
    if df['toss_winner'][ind]==df['team1'][ind]:
        winner_of_toss.append(0)
    else:
        winner_of_toss.append(1)
df.insert(5,'winner_of_toss',winner_of_toss)
df.drop('winner',axis=1,inplace=True)
df.drop('toss_winner',axis=1,inplace=True)
df.head()

,id,city,team1,team2,winner_of_toss,toss_decision,dl_applied,match_winner
0,1.0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,1,field,0.0,0
1,2.0,Pune,Mumbai Indians,Rising Pune Supergiant,1,field,0.0,1
2,3.0,Rajkot,Gujarat Lions,Kolkata Knight Riders,1,field,0.0,1
3,4.0,Indore,Rising Pune Supergiant,Kings XI Punjab,1,field,0.0,1
4,5.0,Bangalore,Royal Challengers Bangalore,Delhi Daredevils,0,bat,0.0,0


In [8]:
df2 = pd.read_csv("teams.csv")
df2.head()

,team1
0,Pune Warriors
1,Kolkata Knight Riders
2,Rajasthan Royals
3,Kochi Tuskers Kerala
4,Gujarat Lions


In [9]:
print(df2.team1.unique())
len(df2.team1.unique())

['Pune Warriors' 'Kolkata Knight Riders' 'Rajasthan Royals'
 'Kochi Tuskers Kerala' 'Gujarat Lions' 'Chennai Super Kings'
 'Rising Pune Supergiants' 'Delhi Daredevils' 'Deccan Chargers'
 'Delhi Capitals' 'Mumbai Indians' 'Sunrisers Hyderabad'
 'Rising Pune Supergiant' 'Royal Challengers Bangalore' 'Kings XI Punjab']


15

In [10]:


for ind in df.index:
    if df['team1'][ind]=="Deccan Chargers":
        df['team1'][ind]='Sunrisers Hyderabad'
        
    if df['team1'][ind]=="Delhi Daredevils":
        df['team1'][ind]='Delhi Capitals'
        
    if df['team2'][ind]=="Deccan Chargers":
        df['team2'][ind]='Sunrisers Hyderabad'
        
    if df['team2'][ind]=="Delhi Daredevils":
        df['team2'][ind]='Delhi Capitals'
        
for ind in df.index:
    if df['team1'][ind]=="Rising Pune Supergiants":
        df['team1'][ind]='Rising Pune Supergiant'
        
    if df['team2'][ind]=="Rising Pune Supergiants":
        df['team2'][ind]='Rising Pune Supergiant'
df.head()

/home/piyush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/piyush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/piyush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoi

,id,city,team1,team2,winner_of_toss,toss_decision,dl_applied,match_winner
0,1.0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,1,field,0.0,0
1,2.0,Pune,Mumbai Indians,Rising Pune Supergiant,1,field,0.0,1
2,3.0,Rajkot,Gujarat Lions,Kolkata Knight Riders,1,field,0.0,1
3,4.0,Indore,Rising Pune Supergiant,Kings XI Punjab,1,field,0.0,1
4,5.0,Bangalore,Royal Challengers Bangalore,Delhi Capitals,0,bat,0.0,0


In [11]:
print(df.team1.unique())
len(df.team1.unique())

['Sunrisers Hyderabad' 'Mumbai Indians' 'Gujarat Lions'
 'Rising Pune Supergiant' 'Royal Challengers Bangalore'
 'Kolkata Knight Riders' 'Delhi Capitals' 'Kings XI Punjab'
 'Chennai Super Kings' 'Rajasthan Royals' 'Kochi Tuskers Kerala'
 'Pune Warriors']


12

    READING DATA FROM "deliveries.csv to add two new features i.e. batting average and average wickets for each TEAM

In [12]:
df3 = pd.read_csv("deliveries.csv")
df3.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [13]:
for ind in df3.index:
    if df3['batting_team'][ind]=="Deccan Chargers":
        df3['batting_team'][ind]='Sunrisers Hyderabad'
        
    if df3['batting_team'][ind]=="Delhi Daredevils":
        df3['batting_team'][ind]='Delhi Capitals'
        
    if df3['bowling_team'][ind]=="Deccan Chargers":
        df3['bowling_team'][ind]='Sunrisers Hyderabad'
        
    if df3['bowling_team'][ind]=="Delhi Daredevils":
        df3['bowling_team'][ind]='Delhi Capitals'
        
for ind in df3.index:
    if df3['batting_team'][ind]=="Rising Pune Supergiants":
        df3['batting_team'][ind]='Rising Pune Supergiant'
        
    if df3['bowling_team'][ind]=="Rising Pune Supergiants":
        df3['bowling_team'][ind]='Rising Pune Supergiant'
df3.head()

/home/piyush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/piyush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/piyush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoi

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


### CALCULATE BATTING AVERAGE FOR EACH TEAM

In [14]:
"""
    Calculating Batting average for each team
"""

group_by_innings_batting_team_sum = df3.groupby(['batting_team'])['total_runs'].sum().reset_index()
print(group_by_innings_batting_team_sum)


counts_of_match_played_by_each_team ={}
li = list(df['team1'].unique())
for i in li:
    counts_of_match_played_by_each_team[i] = 0

for ind in df.index:
    counts_of_match_played_by_each_team[df['team1'][ind]] +=1
    counts_of_match_played_by_each_team[df['team2'][ind]] +=1
print("\n")

group_by_innings_batting_team_sum['average_score'] = 0
for ind in group_by_innings_batting_team_sum.index:
    group_by_innings_batting_team_sum['average_score'][ind] = group_by_innings_batting_team_sum['total_runs'][ind]/counts_of_match_played_by_each_team[group_by_innings_batting_team_sum['batting_team'][ind]]
group_by_innings_batting_team_sum.drop('total_runs',inplace=True,axis=1)
print(group_by_innings_batting_team_sum)

                   batting_team  total_runs
0           Chennai Super Kings       26418
1                Delhi Capitals       27018
2                 Gujarat Lions        4862
3               Kings XI Punjab       27893
4          Kochi Tuskers Kerala        1901
5         Kolkata Knight Riders       27419
6                Mumbai Indians       29809
7                 Pune Warriors        6358
8              Rajasthan Royals       22431
9        Rising Pune Supergiant        4533
10  Royal Challengers Bangalore       28126
11          Sunrisers Hyderabad       28522


                   batting_team  average_score
0           Chennai Super Kings            161
1                Delhi Capitals            154
2                 Gujarat Lions            162
3               Kings XI Punjab            158
4          Kochi Tuskers Kerala            135
5         Kolkata Knight Riders            154
6                Mumbai Indians            159
7                 Pune Warriors            141
8  

/home/piyush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
"""
    Merging batting average score into final_df

"""
final_df = df.merge(group_by_innings_batting_team_sum, left_on='team1', right_on='batting_team',validate='m:1')
final_df.sort_values("id", axis = 0, ascending = True,inplace = True, na_position ='last')
final_df.rename(columns = {'average_score':'average_team1_score'}, inplace = True)
final_df.drop("batting_team",inplace=True,axis = 1)

final_df = final_df.merge(group_by_innings_batting_team_sum, left_on='team2', right_on='batting_team',validate='m:1')
final_df.sort_values("id", axis = 0, ascending = True,inplace = True, na_position ='last')
final_df.rename(columns = {'average_score':'average_team2_score'}, inplace = True)
final_df.drop("batting_team",inplace=True,axis = 1)
final_df.head()

,id,city,team1,team2,winner_of_toss,toss_decision,dl_applied,match_winner,average_team1_score,average_team2_score
0,1.0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,1,field,0.0,0,155,158
94,2.0,Pune,Mumbai Indians,Rising Pune Supergiant,1,field,0.0,1,159,151
109,3.0,Rajkot,Gujarat Lions,Kolkata Knight Riders,1,field,0.0,1,162,154
204,4.0,Indore,Rising Pune Supergiant,Kings XI Punjab,1,field,0.0,1,151,158
289,5.0,Bangalore,Royal Challengers Bangalore,Delhi Capitals,0,bat,0.0,0,158,154


### CALCULATING BOWLING AVERAGE FOREACH TEAM

In [16]:
"""
    Calculating Bowling average for each team
"""

group_by_innings_bowling_team_sum = df3.groupby(['bowling_team'])['player_dismissed'].count().reset_index()
print(group_by_innings_bowling_team_sum)


group_by_innings_bowling_team_sum['average_wickets'] = 0
for ind in group_by_innings_bowling_team_sum.index:
    group_by_innings_bowling_team_sum['average_wickets'][ind] = group_by_innings_bowling_team_sum['player_dismissed'][ind]/counts_of_match_played_by_each_team[group_by_innings_bowling_team_sum['bowling_team'][ind]]
group_by_innings_bowling_team_sum.drop('player_dismissed',inplace=True,axis=1)
print("\n")
print(group_by_innings_bowling_team_sum)


                   bowling_team  player_dismissed
0           Chennai Super Kings              1030
1                Delhi Capitals              1020
2                 Gujarat Lions               151
3               Kings XI Punjab               994
4          Kochi Tuskers Kerala                74
5         Kolkata Knight Riders              1010
6                Mumbai Indians              1138
7                 Pune Warriors               238
8              Rajasthan Royals               856
9        Rising Pune Supergiant               189
10  Royal Challengers Bangalore              1036
11          Sunrisers Hyderabad              1098


                   bowling_team  average_wickets
0           Chennai Super Kings                6
1                Delhi Capitals                5
2                 Gujarat Lions                5
3               Kings XI Punjab                5
4          Kochi Tuskers Kerala                5
5         Kolkata Knight Riders                5
6    

/home/piyush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


### MERGING ALL DATAFRAMES

In [17]:
"""
    Merging Bowling average into dataset.
"""

final_df = final_df.merge(group_by_innings_bowling_team_sum, left_on='team1', right_on='bowling_team',validate='m:1')
final_df.sort_values("id", axis = 0, ascending = True,inplace = True, na_position ='last')
final_df.rename(columns = {'average_wickets':'average_team1_wickets'}, inplace = True)
final_df.head()

final_df.drop("bowling_team",inplace=True,axis = 1)

final_df = final_df.merge(group_by_innings_bowling_team_sum, left_on='team2', right_on='bowling_team',validate='m:1')
final_df.sort_values("id", axis = 0, ascending = True,inplace = True, na_position ='last')
final_df.rename(columns = {'average_wickets':'average_team2_wickets'}, inplace = True)
final_df.drop("bowling_team",inplace=True,axis = 1)
final_df.head()

,id,city,team1,team2,winner_of_toss,toss_decision,dl_applied,match_winner,average_team1_score,average_team2_score,average_team1_wickets,average_team2_wickets
0,1.0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,1,field,0.0,0,155,158,6,5
94,2.0,Pune,Mumbai Indians,Rising Pune Supergiant,1,field,0.0,1,159,151,6,6
109,3.0,Rajkot,Gujarat Lions,Kolkata Knight Riders,1,field,0.0,1,162,154,5,5
204,4.0,Indore,Rising Pune Supergiant,Kings XI Punjab,1,field,0.0,1,151,158,6,5
289,5.0,Bangalore,Royal Challengers Bangalore,Delhi Capitals,0,bat,0.0,0,158,154,5,5


In [18]:
final_df = final_df.where(final_df['team1']!="Kochi Tuskers Kerala")
final_df = final_df.where(final_df['team2']!="Kochi Tuskers Kerala")
final_df = final_df.dropna(axis=0)
final_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Capitals', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Pune Warriors'],
      dtype=object)

### ENCODING OF STRING FEATURES

In [19]:
dummy_teams = pd.get_dummies(final_df,columns=['team1','team2','toss_decision'],prefix = ['t1','t2','toss'])
dummy_teams.head()

,id,city,winner_of_toss,dl_applied,match_winner,average_team1_score,average_team2_score,average_team1_wickets,average_team2_wickets,t1_Chennai Super Kings,...,t2_Kings XI Punjab,t2_Kolkata Knight Riders,t2_Mumbai Indians,t2_Pune Warriors,t2_Rajasthan Royals,t2_Rising Pune Supergiant,t2_Royal Challengers Bangalore,t2_Sunrisers Hyderabad,toss_bat,toss_field
0,1.0,Hyderabad,1.0,0.0,0.0,155.0,158.0,6.0,5.0,0,...,0,0,0,0,0,0,1,0,0,1
94,2.0,Pune,1.0,0.0,1.0,159.0,151.0,6.0,6.0,0,...,0,0,0,0,0,1,0,0,0,1
109,3.0,Rajkot,1.0,0.0,1.0,162.0,154.0,5.0,5.0,0,...,0,1,0,0,0,0,0,0,0,1
204,4.0,Indore,1.0,0.0,1.0,151.0,158.0,6.0,5.0,0,...,1,0,0,0,0,0,0,0,0,1
289,5.0,Bangalore,0.0,0.0,0.0,158.0,154.0,5.0,5.0,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
#dummy_teams.drop("city",axis=1,inplace=True)
dummy_teams.drop("dl_applied",axis=1,inplace=True)
dummy_teams.drop("id",axis=1,inplace=True)
dummy_teams.head()

,winner_of_toss,average_team1_score,average_team2_score,average_team1_wickets,average_team2_wickets,t1_Chennai Super Kings,t1_Delhi Capitals,t1_Gujarat Lions,t1_Kings XI Punjab,t1_Kolkata Knight Riders,...,t2_Kings XI Punjab,t2_Kolkata Knight Riders,t2_Mumbai Indians,t2_Pune Warriors,t2_Rajasthan Royals,t2_Rising Pune Supergiant,t2_Royal Challengers Bangalore,t2_Sunrisers Hyderabad,toss_bat,toss_field
0,1.0,155.0,158.0,6.0,5.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
94,1.0,159.0,151.0,6.0,6.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
109,1.0,162.0,154.0,5.0,5.0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
204,1.0,151.0,158.0,6.0,5.0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
289,0.0,158.0,154.0,5.0,5.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [22]:
y = dummy_teams['match_winner'].values
dummy_teams.drop('match_winner',axis=1,inplace=True)
X = dummy_teams.iloc[:,:].values
print(X)
print(y)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
print(X)


[[1.0000e+00 1.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 [2.0000e+00 1.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 [3.0000e+00 1.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 ...
 [1.1413e+04 1.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 [1.1414e+04 1.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 [1.1415e+04 0.0000e+00 0.0000e+00 ... 0.0000e+00 1.0000e+00 0.0000e+00]]
[0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0.
 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0.
 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1.
 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1.
 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0.
 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1.
 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0.

# STEP2 : TRAINING AND TESTING CLASSIFICATION MODELS TO PREDICT THE MATCH WINNER 

In [26]:
"""
Implementing Random Forest Classification
"""
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
classifier = RandomForestClassifier(n_estimators=1000)
accuracies_random = cross_val_score(estimator = classifier, X = X, y = y, cv = 10)
print("Accuracy: {:.2f} %".format((accuracies_random.mean()*100)))
print("Standard Deviation: {:.2f} %".format(accuracies_random.std()*100))

Accuracy: 69.28 %
Standard Deviation: 8.12 %


In [28]:
"""
Implementing Naive Bayes
"""

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
classifierGB = GaussianNB()
accuracies_GB = cross_val_score(estimator = classifierGB, X = X, y = y, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies_GB.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies_GB.std()*100))

Accuracy: 77.55 %
Standard Deviation: 4.39 %


In [29]:
"""
Implementing SVM
"""


from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
classifierSVC = SVC()
accuraciesSVC = cross_val_score(estimator = classifierSVC, X = X, y = y, cv = 10)
print("Accuracy: {:.2f} %".format(accuraciesSVC.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuraciesSVC.std()*100))

/home/piyush/.local/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/piyush/.local/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/piyush/.local/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/piyush/.local/lib/python3.6/site-packages/sklearn/svm/base

Accuracy: 76.81 %
Standard Deviation: 7.59 %


/home/piyush/.local/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/piyush/.local/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/piyush/.local/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/piyush/.local/lib/python3.6/site-packages/sklearn/svm/base

In [30]:
"""
Implementing Knn
"""


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
classifierknn = KNeighborsClassifier(n_neighbors=200)
accuraciesknn = cross_val_score(estimator = classifierknn, X = X, y = y, cv = 10)
print("Accuracy: {:.2f} %".format(accuraciesknn.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuraciesknn.std()*100))

Accuracy: 75.59 %
Standard Deviation: 4.31 %


# SUMMARY
 ### 1.RANDOM FOREST    : 70%
 ### 2.NAIVE BAYES            : 74%
 ### 3.SVM CLASSIFIER      : 76.81%
 ### 4.KNN CLASSIFIER      : 75.59%